In [ ]:
    from collections import UserList
    from IPython import get_ipython
    from IPython.core.interactiveshell import InteractiveShell
    import CommonMark
    from IPython.core.inputsplitter import IPythonInputSplitter
    from IPython.core.inputtransformer import InputTransformer
    from textwrap import indent

In [ ]:
    def codify(source, parser=CommonMark.Parser, str="""""") -> str:
        """Convert markdown to code with the correct line numbers and positions in a string.

        This function replaces non-code_block nodes with blanklines."""
        for node, _ in parser().parse(source).walker():
            if node.t == 'code_block':
                line, col = node.sourcepos[0]
                end = line + len(node.literal.splitlines())
                lines = node.literal.splitlines()
                while len(str.splitlines()) < (line-1):
                    if not str and str is not 0:
                        str += (' '*col)+'#'
                    str += '\n'
                str += indent(node.literal, ' '*col)
        return str

In [7]:
    class Markdown(UserList, InputTransformer):
        def push(self, object): 
            return self.append(object)
        def load(self):
            ip = get_ipython() or InteractiveShell()
            if ip.input_transformer_manager.physical_line_transforms:
                self.input_transformer_manager = ip.input_transformer_manager
                ip.input_transformer_manager = IPythonInputSplitter(
                    logical_line_transforms=[], physical_line_transforms=[], python_line_transforms=[])
            else:
                self.input_transformer_manager = IPythonInputSplitter()
            ip.input_transformer_manager.python_line_transforms = [
                object for object in ip.input_transformer_manager.python_line_transforms
                if not isinstance(object, Markdown)
            ] + [self]
            return self

        def reset(self):
            str, self.data = '\n'.join(self), []
            if str and str.splitlines()[0].strip():
                from IPython.display import display, Markdown
                display(Markdown(str))
            return self.input_transformer_manager.transform_cell(codify(str))

In [8]:
    from nbconvert.preprocessors import Preprocessor
    class Normalize(Preprocessor):
        def preprocess_cell(Normalize, cell, resources={}, index=0):
            if cell['cell_type'] == 'code':
                cell.source = Markdown(cell['source']).reset()
            return cell, resources

In [9]:
    def load_ipython_extension(ip=get_ipython()): Markdown().load()
    def unload_ipython_extension(ip=get_ipython()):
        for object in ip.input_transformer_manager.python_line_transforms:
            if isinstance(object, Markdown):
                ip.input_transformer_manager = object.input_transformer_manager

In [10]:
    if __name__ == '__main__': 
        __import__('doctest').testmod()
        %load_ext rites
        load_ipython_extension()
        

The rites extension is already loaded. To reload it, use:
  %reload_ext rites
